### Storage Original Dataset

#### 01 - Import dataset and load environment variables

In [1]:
import pandas as pd
import boto3
from env import AWS_PROFILE_NAME, RDS_DB_NAME, RDS_INSTANCE_NAME, RDS_PASSWORD, RDS_USER, DEFAULT_RDS_DB_TABLE

profile_name = AWS_PROFILE_NAME
master_username = RDS_USER
master_password = RDS_PASSWORD
db_name = RDS_DB_NAME
db_instance_identifier = RDS_INSTANCE_NAME
default_table = DEFAULT_RDS_DB_TABLE

boto_session = boto3.Session(profile_name=profile_name)

rds_client = boto_session.client("rds")

df = pd.read_csv("./dataset/hotel-reservations.csv")

df

Booking_ID  no_of_adults  no_of_children  no_of_weekend_nights  \
0       INN00001             2               0                     1   
1       INN00002             2               0                     2   
2       INN00003             1               0                     2   
3       INN00004             2               0                     0   
4       INN00005             2               0                     1   
...          ...           ...             ...                   ...   
36270   INN36271             3               0                     2   
36271   INN36272             2               0                     1   
36272   INN36273             2               0                     2   
36273   INN36274             2               0                     0   
36274   INN36275             2               0                     1   

       no_of_week_nights type_of_meal_plan  required_car_parking_space  \
0                      2       Meal Plan 1                           0   
1                      3      Not Selected                           0   
2                      1       Meal Plan 1                           0   
3                      2       Meal Plan 1                           0   
4                      1      Not Selected                           0   
...                  ...               ...                         ...   
36270                  6       Meal Plan 1                           0   
36271                  3       Meal Plan 1                           0   
36272                  6       Meal Plan 1                           0   
36273                  3      Not Selected                           0   
36274                  2       Meal Plan 1                           0   

      room_type_reserved  lead_time  arrival_year  arrival_month  \
0            Room_Type 1        224          2017             10   
1            Room_Type 1          5          2018             11   
2            Room_Type 1          1          2018              2   
3            Room_Type 1        211          2018              5   
4            Room_Type 1         48          2018              4   
...                  ...        ...           ...            ...   
36270        Room_Type 4         85          2018              8   
36271        Room_Type 1        228          2018             10   
36272        Room_Type 1        148          2018              7   
36273        Room_Type 1         63          2018              4   
36274        Room_Type 1        207          2018             12   

       arrival_date market_segment_type  repeated_guest  \
0                 2             Offline               0   
1                 6              Online               0   
2                28              Online               0   
3                20              Online               0   
4                11              Online               0   
...             ...                 ...             ...   
36270             3              Online               0   
36271            17              Online               0   
36272             1              Online               0   
36273            21              Online               0   
36274            30             Offline               0   

       no_of_previous_cancellations  no_of_previous_bookings_not_canceled  \
0                                 0                                     0   
1                                 0                                     0   
2                                 0                                     0   
3                                 0                                     0   
4                                 0                                     0   
...                             ...                                   ...   
36270                             0                                     0   
36271                             0                                     0   
36272                            

##### 02 - Create RDS instance if not exists

In [2]:
from rds import create_rds_instance

create_rds_instance(
    rds_client,
    db_instance_identifier,
    db_name,
    master_password,
    master_username,
)

# After creation, await a few minutes for the instance to be ready before proceeding to get the endpoint

RDS Instance already exists


##### 03 - Get RDS instance endpoint if created

In [3]:
from rds import get_rds_instance

endpoint = get_rds_instance(rds_client, db_instance_identifier)

RDS Instance founded


##### 04 - Create SQLAlchemy engine with endpoint

In [4]:
from rds import create_rds_engine

engine = create_rds_engine(master_username, master_password, endpoint, db_name)

##### 05 - Insert unmodifield dataset in database

In [5]:
df.to_sql(default_table, con=engine, if_exists="replace", index=False)

36275